In [1]:
pip install pyaudio librosa transformers torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyaudio
import wave

def record_audio(duration, output_filename):
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    fs = 44100  
    # Record at 44100 samples per second

    p = pyaudio.PyAudio()  
    # Create an interface to PortAudio

    print('Recording')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  
    # Initialize array to store frames

    # Store data in chunks for the specified duration
    for i in range(0, int(fs / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    with wave.open(output_filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(sample_format))
        wf.setframerate(fs)
        wf.writeframes(b''.join(frames))

record_audio(10, 'output.wav')  # Record audio for 10 seconds


Recording
Finished recording


In [3]:
import librosa

def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=16000)
    return audio, sr

audio, sr = load_audio('output.wav')


In [4]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

# Load the pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

def speech_to_text(audio, sr):
    inputs = processor(audio, sampling_rate=sr, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription[0]

transcription = speech_to_text(audio, sr)
print(transcription)


D:\anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at 

In [5]:
pip install transformers


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import transformers
print(transformers.__version__)


4.32.1


In [7]:
pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import sentencepiece as spm 
s = spm.SentencePieceProcessor(model_file='spm.model')

IndentationError: unexpected indent (2454787082.py, line 3)

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def summarize_text(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

summary = summarize_text(transcription)
print(summary)


D:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rithe\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


a a a a a a a a a a a a a a a a a a a a.


In [ ]:
if __name__ == "__main__":
    duration = 10  # Duration in seconds
    output_filename = 'output.wav'

    # Step 1: Record Audio
    record_audio(duration, output_filename)

    # Step 2: Load and Preprocess Audio
    audio, sr = load_audio(output_filename)

    # Step 3: Convert Speech to Text
    transcription = speech_to_text(audio, sr)
    print("Transcription:", transcription)

    # Step 4: Summarize Text
    summary = summarize_text(transcription)
    print("Summary:", summary)


In [11]:
pip install wxpython

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for wxpython from https://files.pythonhosted.org/packages/36/60/0e40b653df66328e53c60d69c4f6c835655837e89da42e95feb28aa18118/wxPython-4.2.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/17.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.8 MB 495.5 kB/s eta 0:00:36
    --------------------------------------- 0.4/17.8 MB 3.2 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/17.8 MB 7.3 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/17.8 MB 10.8 MB/s eta 0:00:02
   ------ --------------------------------- 3.0/17.8 MB 11.2 MB/s eta 0:00:02
   ------- -------------------------------- 3.4/17.8 MB 11.5 MB/s eta 0:00:02
   ---------- ----------------------------- 4.7/17.8 MB 13.1 MB/s eta 0:00:02
   ------------ -----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import wx
import threading
import pyaudio
import wave
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the pre-trained models and processors
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = T5Tokenizer.from_pretrained("t5-small")
summary_model = T5ForConditionalGeneration.from_pretrained("t5-small")

class SpeechToTextApp(wx.Frame):
    def __init__(self, *args, **kw):
        super(SpeechToTextApp, self).__init__(*args, **kw)

        self.InitUI()
        self.Bind(wx.EVT_CLOSE, self.OnClose)

    def InitUI(self):
        panel = wx.Panel(self)

        vbox = wx.BoxSizer(wx.VERTICAL)

        # Record Button
        self.record_button = wx.Button(panel, label='Record')
        self.record_button.Bind(wx.EVT_BUTTON, self.OnRecord)
        vbox.Add(self.record_button, flag=wx.EXPAND|wx.LEFT|wx.RIGHT|wx.TOP, border=10)

        # Convert Button
        self.convert_button = wx.Button(panel, label='Convert')
        self.convert_button.Bind(wx.EVT_BUTTON, self.OnConvert)
        self.convert_button.Disable()
        vbox.Add(self.convert_button, flag=wx.EXPAND|wx.LEFT|wx.RIGHT|wx.TOP, border=10)

        # Text Display
        self.text_ctrl = wx.TextCtrl(panel, style=wx.TE_MULTILINE|wx.TE_READONLY)
        vbox.Add(self.text_ctrl, proportion=1, flag=wx.EXPAND|wx.LEFT|wx.RIGHT|wx.TOP, border=10)

        panel.SetSizer(vbox)

        self.SetSize((400, 300))
        self.SetTitle('Speech to Text App')
        self.Centre()

    def OnRecord(self, event):
        self.record_button.Disable()
        self.text_ctrl.SetValue("Recording...")
        threading.Thread(target=self.record_audio, args=(10, 'output.wav')).start()

    def OnConvert(self, event):
        self.convert_button.Disable()
        self.text_ctrl.SetValue("Converting...")
        threading.Thread(target=self.process_audio).start()

    def record_audio(self, duration, output_filename):
        chunk = 1024
        sample_format = pyaudio.paInt16
        channels = 1
        fs = 44100

        p = pyaudio.PyAudio()

        stream = p.open(format=sample_format,
                        channels=channels,
                        rate=fs,
                        frames_per_buffer=chunk,
                        input=True)

        frames = []

        for _ in range(0, int(fs / chunk * duration)):
            data = stream.read(chunk)
            frames.append(data)

        stream.stop_stream()
        stream.close()
        p.terminate()

        with wave.open(output_filename, 'wb') as wf:
            wf.setnchannels(channels)
            wf.setsampwidth(p.get_sample_size(sample_format))
            wf.setframerate(fs)
            wf.writeframes(b''.join(frames))

        wx.CallAfter(self.OnRecordComplete)

    def OnRecordComplete(self):
        self.text_ctrl.SetValue("Recording complete. Click 'Convert' to process the audio.")
        self.convert_button.Enable()

    def process_audio(self):
        audio, sr = librosa.load('output.wav', sr=16000)
        transcription = self.speech_to_text(audio, sr)
        summary = self.summarize_text(transcription)
        wx.CallAfter(self.OnConvertComplete, transcription, summary)

    def speech_to_text(self, audio, sr):
        inputs = processor(audio, sampling_rate=sr, return_tensors="pt", padding=True)
        with torch.no_grad():
            logits = model(inputs.input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
        return transcription[0]

    def summarize_text(self, text):
        inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = summary_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    def OnConvertComplete(self, transcription, summary):
        self.text_ctrl.SetValue(f"Transcription:\n{transcription}\n\nSummary:\n{summary}")
        self.record_button.Enable()
        self.convert_button.Disable()

    def OnClose(self, event):
        self.Destroy()

if __name__ == '__main__':
    app = wx.App()
    ex = SpeechToTextApp(None)
    ex.Show()
    app.MainLoop()
